##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# A simple classification model using Keras with Cloud TPUs


## Overview

This notebook shows how to use Keras to build a simple classification model. The model can train, evaluate, and generate predictions using Cloud TPUs. It uses the iris dataset to predict the species of the flower and also shows how to use your own data instead of using pre-loaded data. This model uses 4 input features ***(SepalLength, SepalWidth, PetalLength, PetalWidth)*** to determine one of these flower species ***(Setosa, Versicolor, Virginica)***.

The model trains for 50 epochs and completes in approximately 2 minutes.

This notebook is hosted on GitHub. To view it in its original repository, after opening the notebook, select **File > View on GitHub**.


**NOTE:** This tutorial is designed to show how to write a simple model using Keras. It should not be used for comparision with training on CPU's because of the very small amount of data being used.

## Learning objectives

In this Colab, you will learn how to:
*   Define a Keras model with 2 hidden layers and 10 nodes in each layer.
*   Create and compile a Keras model on TPU with a distribution strategy.
*   Train, evaluate, and and generate predictions on Cloud TPU.






## Instructions

<h3>  &nbsp;&nbsp;Train on TPU&nbsp;&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a></h3>

   1. On the main menu, click Runtime and select **Change runtime type**. Set "TPU" as the hardware accelerator.
   1. Click Runtime again and select **Runtime > Run All**. You can also run the cells manually with Shift-ENTER. 

### Imports

In [0]:
import json
import os
import pandas as pd
import pprint
import tensorflow as tf
import time
import numpy as np
from tensorflow import keras

## TPUs on the cloud

### Resolve TPU Address

In [0]:
use_tpu = True #@param {type:"boolean"}

if use_tpu:
    assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

if 'COLAB_TPU_ADDR' in os.environ:
  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
else:
  TF_MASTER=''

### FLAGS used as model params




In [0]:
# Model specific parameters

# TPU address
tpu_address = TF_MASTER

# Number of epochs
epochs = 50

# Number of steps_per_epoch
steps_per_epoch = 5

# NOTE: Total number of training steps = Number of epochs * Number of steps_per_epochs

### Set up TPU distribution strategy


In [4]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.86.6.178:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.86.6.178:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


## Data, model, and training

### Download training input data and define prediction input & output

In [5]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

PREDICTION_INPUT_DATA = {
    'SepalLength': [6.9, 5.1, 5.9, 6.0, 5.5, 6.2, 5.5, 6.3],
    'SepalWidth': [3.1, 3.3, 3.0, 3.4, 2.5, 2.9, 4.2, 2.8],
    'PetalLength': [5.4, 1.7, 4.2, 4.5, 4.0, 4.3, 1.4, 5.1],
    'PetalWidth': [2.1, 0.5, 1.5, 1.6, 1.3, 1.3, 0.2, 1.5],
}

PREDICTION_OUTPUT_DATA = ['Virginica', 'Setosa', 'Versicolor', 'Versicolor', 'Versicolor', 'Versicolor', 'Setosa', 'Virginica']

def maybe_download():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

    return train_path, test_path

def load_data(y_name='Species'):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
    train_path, test_path = maybe_download()

    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0, dtype={'SepalLength': pd.np.float32,
        'SepalWidth': pd.np.float32, 'PetalLength': pd.np.float32, 'PetalWidth': pd.np.float32, 'Species': pd.np.int32})
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0, dtype={'SepalLength': pd.np.float32,
        'SepalWidth': pd.np.float32, 'PetalLength': pd.np.float32, 'PetalWidth': pd.np.float32, 'Species': pd.np.int32})
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y), (test_x, test_y)

# Fetch the data
(train_x, train_y), (test_x, test_y) = load_data()

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


We have 4 features: SepalWidth, SepalLength, PetalWidth, and PetalLength.

In [6]:
print(train_x.shape)
print(test_x.shape)

(120, 4)
(30, 4)


We predict the type of flower.

In [7]:
print(train_y.shape)
print(test_y.shape)

(120,)
(30,)


### Define a Keras model (2 hidden layers with 10 neurons in each)

In [0]:
def get_model():
  return keras.Sequential([
    keras.layers.Dense(10, input_shape=(4,), activation=tf.nn.relu, name = "Dense_1"),
    keras.layers.Dense(10, activation=tf.nn.relu, name = "Dense_2"),
    keras.layers.Dense(3, activation=None, name = "logits"),
    keras.layers.Dense(3, activation=tf.nn.softmax, name = "softmax")
  ])

### Compiling the model with a distribution strategy
To make the model usable by a TPU, we first must create and compile it using a distribution strategy.

In [9]:
with strategy.scope():
  model = get_model()
  model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1), 
                loss='sparse_categorical_crossentropy',
                metrics=['sparse_categorical_crossentropy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Dense_1 (Dense)              (None, 10)                50        
_________________________________________________________________
Dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
logits (Dense)               (None, 3)                 33        
_________________________________________________________________
softmax (Dense)              (None, 3)                 12        
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________


### Train the model on TPU

In [10]:
# Train the model
model.fit(
  train_x.values, train_y.values,
  steps_per_epoch = steps_per_epoch,
  epochs=epochs,
)

Epoch 1/50
5/5 [==============================] - 0s 7ms/step - sparse_categorical_crossentropy: 0.9614 - loss: 0.9614
Epoch 2/50
5/5 [==============================] - 0s 6ms/step - sparse_categorical_crossentropy: 0.6416 - loss: 0.6416
Epoch 3/50
5/5 [==============================] - 0s 6ms/step - sparse_categorical_crossentropy: 0.4634 - loss: 0.4634
Epoch 4/50
5/5 [==============================] - 0s 6ms/step - sparse_categorical_crossentropy: 0.5063 - loss: 0.5063
Epoch 5/50
5/5 [==============================] - 0s 6ms/step - sparse_categorical_crossentropy: 0.4151 - loss: 0.4151
Epoch 6/50
5/5 [==============================] - 0s 6ms/step - sparse_categorical_crossentropy: 0.3834 - loss: 0.3834
Epoch 7/50
5/5 [==============================] - 0s 9ms/step - sparse_categorical_crossentropy: 0.3685 - loss: 0.3685
Epoch 8/50
5/5 [==============================] - 0s 6ms/step - sparse_categorical_crossentropy: 0.3720 - loss: 0.3720
Epoch 9/50
5/5 [==============================] 

### Evaluation of the model

In [11]:
model.evaluate(test_x.values, test_y.values,
    batch_size=8)

4/4 [==============================] - 0s 11ms/step - sparse_categorical_crossentropy: 0.0781 - loss: 0.0781


[0.07808293402194977, 0.07808293402194977]

### Save the model

In [0]:
model.save_weights('./DNN_TPU_1024.h5', overwrite=True)

## Prediction

### Prediction data

In [13]:
COLUMNS_NAME=['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
data = pd.DataFrame(PREDICTION_INPUT_DATA, columns=COLUMNS_NAME)
print(data)

   SepalLength  SepalWidth  PetalLength  PetalWidth
0          6.9         3.1          5.4         2.1
1          5.1         3.3          1.7         0.5
2          5.9         3.0          4.2         1.5
3          6.0         3.4          4.5         1.6
4          5.5         2.5          4.0         1.3
5          6.2         2.9          4.3         1.3
6          5.5         4.2          1.4         0.2
7          6.3         2.8          5.1         1.5


### Prediction on CPU

In [14]:
cpu_model = get_model()
cpu_model.load_weights('./DNN_TPU_1024.h5')
cpu_predictions = cpu_model.predict(data)
template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')
for pred_dict, expec in zip(cpu_predictions, PREDICTION_OUTPUT_DATA):
  class_index = np.argmax(pred_dict)
  class_probability = np.max(pred_dict)
  print(template.format(SPECIES[class_index], 100*class_probability, expec))


Prediction is "Virginica" (91.5%), expected "Virginica"

Prediction is "Setosa" (99.8%), expected "Setosa"

Prediction is "Versicolor" (97.2%), expected "Versicolor"

Prediction is "Versicolor" (95.5%), expected "Versicolor"

Prediction is "Versicolor" (96.5%), expected "Versicolor"

Prediction is "Versicolor" (98.7%), expected "Versicolor"

Prediction is "Setosa" (100.0%), expected "Setosa"

Prediction is "Versicolor" (66.1%), expected "Virginica"


We did pretty well predicting on this small sample!